# Sparse Non-Negative PARAFAC

This notebook is based on [sparse_demo.ipynb](sparse_demo.ipynb#parafac). 

As before, we start with a random sparse tensor, constructed so that it has a tensor factorization of rank 5.

Because non-negative PARAFAC can take longer to converge than non-masked PARAFAC and also produce dense factors, we will use a smaller tensor than in the other notebook.

In [1]:
shape = (1000, 1001, 1002)
rank = 5

import sparse
starting_weights = sparse.ones(rank)
starting_factors = [sparse.random((i, rank)) for i in shape]
starting_factors

[<COO: shape=(1000, 5), dtype=float64, nnz=50, fill_value=0.0>,
 <COO: shape=(1001, 5), dtype=float64, nnz=50, fill_value=0.0>,
 <COO: shape=(1002, 5), dtype=float64, nnz=50, fill_value=0.0>]

In [2]:
from tensorly.contrib.sparse.kruskal_tensor import kruskal_to_tensor
tensor = kruskal_to_tensor((starting_weights, starting_factors))
tensor

Format,coo
Data Type,float64
Shape,"(1000, 1001, 1002)"
nnz,4993
Density,4.978055876259469e-06
Read-only,True
Size,156.0K
Storage ratio,0.0


In [3]:
def format_size(size_bytes):
    size = size_bytes
    for unit in ['B', 'KiB', 'MiB', 'GiB', 'TiB']:
        if not int(size/1024):
            return f'{round(size)}.{unit}'
        else:
            size /= 1024

In [4]:
format_size(tensor.nbytes)             # Actual memory usage in GB

'156.KiB'

In [5]:
import numpy as np
format_size(np.prod(tensor.shape) * 8)  # Memory usage if array was dense, in GB

'7.0.GiB'

Now we factor the tensor. Note that at this time, you have to use the `non_negative_parafac` function from the sparse backend when using a sparse mask to avoid memory blowups.

In [6]:
import time
%load_ext memory_profiler
from tensorly.contrib.sparse.decomposition import non_negative_parafac

In [7]:
%%memit
start_time = time.time()
sparse_kruskal = non_negative_parafac(tensor, rank=rank, init='random', verbose=True)
end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))

reconstruction error=0.8951445796395032
iteration 1, reconstraction error: 0.6406660337487468, decrease = 0.25447854589075647
iteration 2, reconstraction error: 0.43831537662673414, decrease = 0.2023506571220126
iteration 3, reconstraction error: 0.40509622332965045, decrease = 0.03321915329708369
iteration 4, reconstraction error: 0.4023990562014931, decrease = 0.002697167128157374
iteration 5, reconstraction error: 0.4008966487263428, decrease = 0.0015024074751502914
iteration 6, reconstraction error: 0.40005130320432714, decrease = 0.0008453455220156503
iteration 7, reconstraction error: 0.39956418126088705, decrease = 0.0004871219434400831
iteration 8, reconstraction error: 0.3992726295545541, decrease = 0.0002915517063329376
iteration 9, reconstraction error: 0.39908990034777664, decrease = 0.00018272920677747662
iteration 10, reconstraction error: 0.39896975652235933, decrease = 0.00012014382541730706
iteration 11, reconstraction error: 0.398887163271864, decrease = 8.25932504953

Let's look at one of the values that was masked out.

In [8]:
tensor.coords.T[0]

array([39, 14, 11])

In [9]:
orig_val = tensor[tuple(tensor.coords.T[0])]
orig_val

0.0007553350336627013

See the [sparse_demo.ipynb](sparse_demo.ipynb) for how to calculate individual values from the factors. Note that we do not compare the entire tensor because it would be dense, and memory usage would be prohibitive.

In [10]:
weights, factors = sparse_kruskal
computed_val = np.sum(np.prod(sparse.stack([factors[i][idx] for i, idx in enumerate(tuple(tensor.coords.T[0]))], 0), 0))
computed_val

0.0007553350336627012

In [11]:
np.abs(orig_val - computed_val)

1.0842021724855044e-19

In [12]:
[f.density for f in factors]

[1.0, 1.0, 1.0]